In [1]:
from mia_models import train_target_model, load_shadow_data, train_shadow_models
from mia_models import prepare_attack_test_data, prep_validation_data, prep_attack_train_data
from mia_models import shokri_attack, prety_print_result
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

2022-09-15 15:53:08.045388: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-15 15:53:08.045414: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [3]:
dataset_name = 'location'
train_size = 840
attack_test_size = 600
test_start = 840

loc_data = pd.read_csv('data/bangkok', na_values=["?"], header=None)
target_dataset = loc_data.sample(n = 1200, replace = False)
df_rest = loc_data.loc[~loc_data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 2000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [4]:
# trian target model
per_class_sample=40
channel=1   
epochs=100
act_layer=6
n_class = 31
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, act_layer, n_class, train_size, test_start, is_synthetic)

2022-09-15 15:53:10.722753: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-15 15:53:10.722794: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (msnkhan-desktop): /proc/driver/nvidia/version does not exist
2022-09-15 15:53:10.723150: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



 Target Train acc :  100.0 Target Test acc :  48.61111044883728


In [5]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 1200

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  99.88095164299011 Shadow Test acc :  50.27777552604675
12/12 [==============================] - 0s 2ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  95.59524059295654 Shadow Test acc :  41.111111640930176
12/12 [==============================] - 0s 2ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  100.0 Shadow Test acc :  43.88888776302338
12/12 [==============================] - 0s 2ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  100.0 Shadow Test acc :  46.11110985279083
12/12 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  100.0 Shadow Test acc :  50.0
12/12 [==============================] - 0s 2ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  100.0 Shadow Test acc :  45.55555582046509
12/12 [==============================] - 0s 2ms/step
Shadow Model  6
Training shadow model 6
Shadow Train

In [6]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

19/19 [==============================] - 0s 2ms/step
1
1/1 [==============================] - 0s 19ms/step
2
1/1 [==============================] - 0s 21ms/step
3
1/1 [==============================] - 0s 21ms/step
4
1/1 [==============================] - 0s 23ms/step
5
1/1 [==============================] - 0s 18ms/step
6
1/1 [==============================] - 0s 26ms/step
7
1/1 [==============================] - 0s 25ms/step
8
2/2 [==============================] - 0s 2ms/step
9
1/1 [==============================] - 0s 20ms/step
10
1/1 [==============================] - 0s 20ms/step
11
1/1 [==============================] - 0s 21ms/step
12
1/1 [==============================] - 0s 22ms/step
13
1/1 [==============================] - 0s 21ms/step
14
1/1 [==============================] - 0s 23ms/step
15
1/1 [==============================] - 0s 50ms/step
16
1/1 [==============================] - 0s 23ms/step
17
1/1 [==============================] - 0s 18ms/step
18
1/1 [==============

In [ ]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [13]:
# synthetic dataset
train_size = 840
attack_test_size = 600
test_start = 840
org_dataset = pd.read_csv('data/bangkok', na_values=["?"], header=None)
target_dataset = pd.read_csv('data/loc_sds_cac_2500.csv', na_values=["?"], header=None).sample(n = 1200, replace = False)
df = org_dataset.iloc[2500:,:]
shadow_dataset = df.sample(n = 1900, replace = False)
df_rest = df.loc[~df.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members = org_dataset.iloc[:2500,:]
attack_test_members = attack_test_members.sample(n=attack_test_size, replace=False)

In [18]:
# trian target model
per_class_sample=40
channel=1   
epochs=100
act_layer=6
n_class = 31
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, act_layer, n_class, train_size, test_start, is_synthetic)


 Target Train acc :  100.0 Target Test acc :  67.22221970558167


In [15]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 1200

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  100.0 Shadow Test acc :  44.44444477558136
12/12 [==============================] - 0s 2ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  100.0 Shadow Test acc :  50.55555701255798
12/12 [==============================] - 0s 5ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  100.0 Shadow Test acc :  46.94444537162781
12/12 [==============================] - 0s 2ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  95.83333134651184 Shadow Test acc :  41.94444417953491
12/12 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  100.0 Shadow Test acc :  46.11110985279083
12/12 [==============================] - 0s 5ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  100.0 Shadow Test acc :  44.72222328186035
12/12 [==============================] - 0s 2ms/step
Shadow Model  6
Training shadow model 6
Shadow Train

In [16]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

19/19 [==============================] - 0s 2ms/step
1
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 23ms/step
3
1/1 [==============================] - 0s 20ms/step
4
1/1 [==============================] - 0s 19ms/step
5
1/1 [==============================] - 0s 20ms/step
6
1/1 [==============================] - 0s 19ms/step
7
1/1 [==============================] - 0s 23ms/step
8
2/2 [==============================] - 0s 2ms/step
9
1/1 [==============================] - 0s 19ms/step
10
1/1 [==============================] - 0s 21ms/step
11
1/1 [==============================] - 0s 19ms/step
12
1/1 [==============================] - 0s 19ms/step
13
1/1 [==============================] - 0s 23ms/step
14
1/1 [==============================] - 0s 22ms/step
15
1/1 [==============================] - 0s 20ms/step
16
1/1 [==============================] - 0s 21ms/step
17
1/1 [==============================] - 0s 19ms/step
18
1/1 [==============

In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#